In [1]:
import json

file_path = "../../external_data/German/german_twitter_2019_2020/german-tweet-sample-2019-04/recording-2019-03-31T23-22.json"
with open(file_path) as f:
    data = json.load(f)

In [3]:
data[1]

{'type': 'quote',
 'id': '1112437933187858432',
 'user': '2457468409',
 'created_at': '2019-03-31T19:34:06+00:00',
 'recorded_at': '2019-03-31T19:35:44.797100+00:00',
 'hashtags': [],
 'urls': ['https://t.co/QFLZltDzhM'],
 'mentions': [],
 'mentioned_ids': [],
 'text': 'um nenémm https://t.co/QFLZltDzhM',
 'refers_to': '1112433027853025282'}

In [13]:
count = 0
count_tag = 0
for sample in data:
    if sample['type'] == 'status':
        count += 1
        if sample['text'].startswith("@"):
            count_tag += 1

In [4]:
all_types = {}

for sample in data:
    if sample['type'] not in all_types:
        all_types[sample['type']] = 0
    else:
        all_types[sample['type']] += 1

In [14]:
count_tag

63

In [5]:
all_types

{'quote': 1355, 'user': 10654, 'status': 4560, 'retweet': 4755, 'reply': 3672}

In [15]:
def extract_data_from_json(file_path):
    with open(file_path) as f:
        data = json.load(f)

    for sample in data:
        if sample['type'] in ['status', 'reply']:
            yield {'type': sample['type'], 'text': sample['text']}

In [21]:
from pathlib import Path
import os
from tqdm.auto import tqdm

base_path = Path("../../external_data/German/german_twitter_2019_2020")
output_path = "../../external_data/German/german_twitter_2019_2020/german_tweet_2019_2020.jsonl"

json_files = []
for root, dirs, files in os.walk(base_path):
    if '__MACOSX' in root:
        continue
    
    for file in files:
        if file.endswith('.json'):
            json_files.append(os.path.join(root, file))

with open(output_path, 'w', encoding='utf-8') as outf:
    for file_path in tqdm(json_files):
        for sample in extract_data_from_json(file_path):
            json.dump(sample, outf, ensure_ascii=False)
            outf.write('\n')

100%|██████████| 312/312 [01:35<00:00,  3.28it/s]


In [1]:
from datasets import load_dataset

german_twitter_dataset = load_dataset("json",
    data_files="../../external_data/German/german_twitter_2019_2020/german_tweet_2019_2020.jsonl",
    split="train"
)

Generating train split: 2426884 examples [00:00, 3163362.27 examples/s]


In [2]:
german_twitter_dataset

Dataset({
    features: ['type', 'text'],
    num_rows: 2426884
})

In [6]:
from collections import Counter

Counter(german_twitter_dataset['type'])

Counter({'status': 1495901, 'reply': 930983})

In [9]:
total_samples = 99_000  # Choose your desired total size
reply_samples = int(total_samples * 0.8)  # 8000
status_samples = int(total_samples * 0.2)  # 2000

# Separate the dataset by type
reply_data = german_twitter_dataset.filter(lambda x: x['type'] == 'reply')
status_data = german_twitter_dataset.filter(lambda x: x['type'] == 'status')

# Sample from each
reply_subset = reply_data.shuffle(seed=42).select(range(min(reply_samples, len(reply_data))))
status_subset = status_data.shuffle(seed=42).select(range(min(status_samples, len(status_data))))

# Combine them
from datasets import concatenate_datasets
final_dataset = concatenate_datasets([reply_subset, status_subset])

# Shuffle the combined dataset
final_dataset = final_dataset.shuffle(seed=42)

In [10]:
final_dataset

Dataset({
    features: ['type', 'text'],
    num_rows: 99000
})

In [11]:
final_dataset.to_parquet("../../external_data/German/german_twitter_2019_2020/german_tweet_samples.parquet")

Creating parquet from Arrow format: 100%|██████████| 99/99 [00:01<00:00, 65.69ba/s]


16167919